# Interactive spec creation

This notebook is intended to make it easier to generate ground truth for trips to put into the evaluation spec files. Putting in a bunch of trips into a spec file and trying to fill in all of them leads to wasted time, specially if we have a lot of relations, since we must query OSM over and over to pull the data to populate the coordinates. And then we have to visualize all the trips at once as part of the validation, so if we find any errors, we need to regenerate the coordinates for all trips.

This notebook allows us to interactively develop individual specs before copy-pasting them into the overall spec file.

### These initial cells are for setup and should not need to be changed

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

#### Let's see what we can do with the OSM API directly

In [ ]:
import autofill_eval_spec as aes

In [ ]:
import folium
import folium.features as fof
import folium.utilities as ful
import branca.element as bre

In [ ]:
import polyline as pl

In [ ]:
import osmapi

In [ ]:
osm = osmapi.OsmApi()

In [ ]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

In [ ]:
def get_marker(loc, disp_color):
    if loc["geometry"]["type"] == "Point":
        curr_latlng = lonlat_swap(loc["geometry"]["coordinates"])
        return folium.Marker(curr_latlng, icon=folium.Icon(color=disp_color),
                  popup="%s" % loc["properties"]["name"])
    elif loc["geometry"]["type"] == "Polygon":
        assert len(loc["geometry"]["coordinates"]) == 1,\
            "Only simple polygons supported!"
        curr_latlng = [lonlat_swap(c) for c in loc["geometry"]["coordinates"][0]]
        print("Returning polygon for %s" % curr_latlng)
        return folium.PolyLine(curr_latlng, color=disp_color, fill=disp_color,
                  popup="%s" % loc["properties"]["name"])        

In [ ]:
def get_map_for_evaluation_trip(trip):
    curr_map = folium.Map()
    get_marker(trip["start_loc"], "green").add_to(curr_map)
    get_marker(trip["end_loc"], "red").add_to(curr_map)
    # trips from relations won't have waypoints
    if "waypoint_coords" in trip:
        for i, wpc in enumerate(trip["waypoint_coords"]):
            folium.map.Marker(
                lonlat_swap(wpc), popup="%d" % i,
                icon=fof.DivIcon(class_name='leaflet-div-icon')).add_to(curr_map)
    print("Found %d coordinates for the route" % (len(trip["route_coords"])))
    latlng_route_coords = [lonlat_swap(rc) for rc in trip["route_coords"]["geometry"]["coordinates"]]
    folium.PolyLine(latlng_route_coords,
                    popup="%s: %s" % (trip["mode"], trip["name"])).add_to(curr_map)
    for i, c in enumerate(latlng_route_coords):
        folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(curr_map)
    curr_map.fit_bounds(ful.get_bounds(trip["route_coords"]["geometry"]["coordinates"], lonlat=True))    
    return curr_map

# This is the important part

Instructions:
- Edit the cell below to include the spec that you are experimenting with
- Run the cells below that to fill it in and display it
- Tweak the various fill/population methods until the trip looks right
- Copy/paste the leg spec into your spec file
- Rinse and repeat

In [ ]:
curr_leg_spec = \
        {
            "id": "unexpected_bike_to_bus",
            "name": "Bike to bus when stop changed due to Thursday night live",
            "mode": "BICYCLING",
            "start_loc": {
                "osm_id": 6129383336,
                "name": "Mountain View City Hall"
            },
            "end_loc": {
                "osm_id": 726963272,
                "name": "El Camino and Miramonte (alternate stop)"                
            },
            "waypoint_coords": [
          [
            -122.0817732810974,
            37.38932665304395
          ],
          [
            -122.08233386278151,
            37.38833995211305
          ],
          [
            -122.08664685487746,
            37.39046251514165
          ],
          [
            -122.08813816308974,
            37.38838683668442
          ],
          [
            -122.08862364292143,
            37.38818438037076
          ]
            ]
        }

In [ ]:
import importlib

In [ ]:
importlib.reload(aes)

In [ ]:
curr_leg_spec = \
        {
            "id": "suburb_bicycling",
            "name": "Los Altos Library to Mountain View Library",
            "mode": "BICYCLING",
            "start_loc":     {
      "type": "Feature",
      "properties": {
          "name": "Los Altos Library"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.11386322975159,
              37.38049489802194
            ],
            [
              -122.11191058158873,
              37.37965088242338
            ],
            [
              -122.11124002933502,
              37.37964235696482
            ],
            [
              -122.11118102073668,
              37.37997911184045
            ],
            [
              -122.11119711399078,
              37.38072934511354
            ],
            [
              -122.11176574230194,
              37.3807208197776
            ],
            [
              -122.11254358291625,
              37.38109167099526
            ],
            [
              -122.11386322975159,
              37.38109593364228
            ],
            [
              -122.11386322975159,
              37.38049489802194
            ]
          ]
        ]
      }
    },
            "end_loc":     {
      "type": "Feature",
      "properties": {
        "name": "Mountain View Library"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.08355963230133,
              37.39091642895306
            ],
            [
              -122.08428382873535,
              37.38975713188671
            ],
            [
              -122.08383858203888,
              37.389573859018185
            ],
            [
              -122.08340406417847,
              37.390196132517175
            ],
            [
              -122.08311975002289,
              37.39012793841312
            ],
            [
              -122.08280861377716,
              37.390656441096894
            ],
            [
              -122.08355426788331,
              37.39096757399895
            ],
            [
              -122.08355963230133,
              37.39091642895306
            ]
          ]
        ]
      }
    },
    
            "polyline": "ayecF~nchVrAp@\\RhB~@}AhEEJ{@~BY^UXSZbB`BpBlArAp@dB|@fAXPF^LRH^Dl@@r@EZ?HH@f@Qh@Ql@TJHPP`@DVBHEXQ@EABh@v@P`@H@@d@`@f@FxAAElZJ^FT?rEYAoAB@X^v@RRTTfChCxD~CxAxA~DhDbAbABBb@`@|BzBFFMRLR?lEFH@nCBnEA|DA~BCxAA~@AxFBVLZCvLAhEGRHJ?t@?dCf@?AnCD@TTNB^ALCHCFT?rC"
        }

In [ ]:
aes.validate_and_fill_leg(curr_leg_spec)

In [ ]:
# curr_leg_spec

In [ ]:
get_map_for_evaluation_trip(curr_leg_spec)